# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

## Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_path = "../WeatherPy/Output/weather_data.csv" 
weather_data = pd.read_csv(weather_data_path)
weather_data

,City Name,Country,Date,Latitude,Longitude,Temperature(F),Humidity(%),Cloudiness(%),Wind Speed(mph)
0,lavrentiya,RU,2021-04-21 14:54:09,65.5833,-171.0000,23.45,69,100,6.67
1,hilo,US,2021-04-21 14:54:09,19.7297,-155.0900,75.20,73,90,3.31
2,suntar,RU,2021-04-21 14:54:10,62.1444,117.6319,27.37,81,100,8.61
3,half moon bay,US,2021-04-21 14:54:11,37.4636,-122.4286,53.01,87,90,3.44
4,la ceiba,HN,2021-04-21 14:54:11,15.7597,-86.7822,73.40,94,75,2.30
...,...,...,...,...,...,...,...,...,...
815,kushmurun,KZ,2021-04-21 14:56:20,52.4503,64.6311,50.47,37,0,8.01
816,matara,LK,2021-04-21 14:56:20,5.9485,80.5353,85.15,68,100,10.47
817,bogo,PH,2021-04-21 14:56:20,11.0517,124.0055,88.86,58,100,5.28
818,zlitan,LY,2021-04-21 14:56:20,32.4674,14.5687,64.80,40,5,16.67


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store 'Latitude' and 'Longitude' into variable locations 
locations = weather_data[['Latitude', 'Longitude']].astype(float)

# convert Humidity into float and store in humidity_perc
humidity_perc = weather_data["Humidity(%)"].astype(float)

# Access maps with unique API key
gmaps.configure(api_key=g_key)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_perc, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#Wow, much heatmap, so humid.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Conditions: 
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

ideal_temp = weather_data['Temperature(F)'].between(70, 80, inclusive = True)
ideal_wind = weather_data['Wind Speed(mph)'] < 10.00
ideal_cloud = weather_data['Cloudiness(%)'] == 0

idealness = {'City Name': weather_data['City Name'], 'Ideal Temp': ideal_temp, 'Ideal Wind': ideal_wind, 'Ideal Cloud': ideal_cloud}
idealness_data = pd.DataFrame(idealness)

ideal_vacay = pd.merge(weather_data, idealness_data, on = 'City Name', how= 'right')

ideal_vacay = ideal_vacay[(ideal_vacay['Ideal Temp'] == True) & (ideal_vacay['Ideal Wind'] == True) & (ideal_vacay['Ideal Cloud'] == True)] 
ideal_vacay #Got the ideal Vacation locations

,City Name,Country,Date,Latitude,Longitude,Temperature(F),Humidity(%),Cloudiness(%),Wind Speed(mph),Ideal Temp,Ideal Wind,Ideal Cloud
114,lakki marwat,PK,2021-04-21 14:54:30,32.6079,70.9114,78.62,38,0,4.12,True,True,True
209,jinchang,CN,2021-04-21 14:53:54,38.4953,102.1739,70.66,14,0,9.60,True,True,True
211,boa vista,BR,2021-04-21 14:50:03,2.8197,-60.6733,77.00,78,0,6.91,True,True,True
260,faya,SA,2021-04-21 14:54:50,18.3851,42.4509,73.40,17,0,8.05,True,True,True
297,hondo,JP,2021-04-21 14:54:56,32.4500,130.2000,78.80,25,0,3.44,True,True,True
332,nisia floresta,BR,2021-04-21 14:55:03,-6.0911,-35.2086,77.00,88,0,1.14,True,True,True
355,cayenne,GF,2021-04-21 14:55:06,4.9333,-52.3333,78.01,100,0,6.26,True,True,True
359,touros,BR,2021-04-21 14:55:06,-5.1989,-35.4608,75.20,94,0,2.30,True,True,True
451,tateyama,JP,2021-04-21 14:55:23,34.9833,139.8667,73.00,60,0,5.75,True,True,True
703,nagato,JP,2021-04-21 14:56:01,34.3833,131.2000,71.60,46,0,3.44,True,True,True


It does not seem like there are a lot of good vacation spots that meets all the weather criteria.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotels = {'City': ideal_vacay['City Name'], 'Country': ideal_vacay['Country']}
hotel_df = pd.DataFrame(hotels)


params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use Longitude and Latitude from ideal_vacay DataFrame
for index, row in ideal_vacay.iterrows():
        lati = row["Latitude"]
        long = row["Longitude"]
        city_name = row['City Name']
        # change location each iteration while leaving original params in place
        params["location"] = f"{lati},{long}"

        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # make request and print url
        print(f'Finding Hotel for {index}: {city_name}')
        hotel_address = requests.get(base_url, params=params).json()
        results = hotel_address['results']
        try:
            print(f"Closest Hotel for {city_name} is {results[0]['name']}.")
            print(f'____________________________')
            hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
            hotel_df.loc[index, 'Address'] = results[0]['vicinity']
            hotel_df.loc[index, 'Latitude'] = results[0]['geometry']['location']['lat']
            hotel_df.loc[index, 'Longitude'] = results[0]['geometry']['location']['lng']
        except (KeyError, IndexError):
            print(f'No Hotels found for {index}: {city_name} ')
            print(f'____________________________')
        

Finding Hotel for 114: lakki marwat
No Hotels found for 114: lakki marwat 
____________________________
Finding Hotel for 209: jinchang
No Hotels found for 209: jinchang 
____________________________
Finding Hotel for 211: boa vista
Closest Hotel for boa vista is Hotel Orla do Rio Branco.
____________________________
Finding Hotel for 260: faya
No Hotels found for 260: faya 
____________________________
Finding Hotel for 297: hondo
Closest Hotel for hondo is Hotel Alegria Gardens Amakusa.
____________________________
Finding Hotel for 332: nisia floresta
Closest Hotel for nisia floresta is Mardunas Hotel & Eventos.
____________________________
Finding Hotel for 355: cayenne
Closest Hotel for cayenne is Mercure Cayenne Royal Amazonia.
____________________________
Finding Hotel for 359: touros
Closest Hotel for touros is Hotel Vila Gale Touros.
____________________________
Finding Hotel for 451: tateyama
Closest Hotel for tateyama is Kagamigaura Onsen Rokuza.
____________________________

In [13]:
hotel_df = hotel_df.dropna(how='any') # dropping the ideal vacay spot without any hotel
hotel_df # taking a peek at the cleaned hotel_df

,City,Country,Hotel Name,Address,Latitude,Longitude
211,boa vista,BR,Hotel Orla do Rio Branco,"R. Castelo Branco, 637 - Calunga, Boa Vista",2.808403,-60.670938
297,hondo,JP,Hotel Alegria Gardens Amakusa,"Amakusa, Hondomachi Hirose, 996",32.472897,130.206904
332,nisia floresta,BR,Mardunas Hotel & Eventos,"Estrada da Lagoa do Bonfim, 5000 - Lagoa do Bo...",-6.045971,-35.219094
355,cayenne,GF,Mercure Cayenne Royal Amazonia,"45 Rue de l'Ara Bleu, Cayenne",4.934771,-52.304552
359,touros,BR,Hotel Vila Gale Touros,"Fazenda das Garças - Gameleira Zona Rural de, ...",-5.226640,-35.415863
451,tateyama,JP,Kagamigaura Onsen Rokuza,"1672-88 Nago, Tateyama",35.023006,139.854047
703,nagato,JP,KULABO Taishokan,"1904-1, Nagato",34.369381,131.182714
790,kerman,IR,Hotel Jahangardy,Kerman,30.284075,57.033726
813,nushki,PK,Nabi Bux Hotel,Nushki,29.554839,66.020150


Upon narrowing down to the 5KM radius of the City coordinates, cities -lakki marwat, jinchang, faya, tessalit did not return a result for type 'lodging' and keyword 'hotel'. Hence, we drop those vacation destination as camping is not an option.

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hlocations = hotel_df[['Latitude', "Longitude"]]

In [15]:
# Add marker layer ontop of heat map

#original heatmap for humidity

gmaps.configure(api_key=g_key)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_perc, 
                                 dissipating=False, max_intensity=100,point_radius = 1)
# Create a hotel map marker
marker_layer = gmaps.marker_layer(hlocations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

Observing the location of the hotel markers on the Humidity heatmap, 
one that particularly stood out is the Kagamigaura Onsen Rokuza in Tateyama, Japan.
The area is very humid and yet fulfils the other conditions of a perfect vacation destination. 
The suggested Hotel is also a Hot Spring hotel which fits into an ideal vacation.
Zooming out from Tateyama, surprisingly, there are another 2 ideal vacation spots, Nagato and Hondo which are also in Japan. 
Compared to Tateyama, they are less humid (Due to the less intense heatmap and also they are not hotsprings)

Moving to the West, we have another 2 ideal vacation destinations located in West Asia, Pakistan and Iran. 
Kerman and Nushki are two location with the identical humidity percentage, 18 and 13. So it is safe to say that this is an ideal spot for those who are suffering from allergies and asthma. 

Next, we travel even further to the west to the South America continent. 
The spots at South America all have over 75% humidity - Boa Vista(78), Cayenne(100), Nisia Floresta(88), Touros(94).
Looking at the locations, these 4 vacation spots are really close to the Equator.
While it may be tempting to book a flight to these spots to get some sun while enjoying the perfect weather, do remember
to stay hydrated as high humidity hinders our human bodies to dissipate heat.
